In [ ]:
import tensorflow as tf
import json

# Update the path to your model saved in the native Keras format.
model_path = "gs://tf_models_cv/v1_Feb_09_2025.keras"

# Load the model
model = tf.keras.models.load_model(model_path)

# Print a summary of the model architecture.
print("=== Model Summary ===")
model.summary()

# Print the model configuration as a JSON.
print("\n=== Model Configuration ===")
config = model.get_config()
print(json.dumps(config, indent=2))

# If the optimizer was saved, print its configuration.
if hasattr(model, "optimizer") and model.optimizer is not None:
    print("\n=== Optimizer Configuration ===")
    optimizer_config = model.optimizer.get_config()
    print(json.dumps(optimizer_config, indent=2))


In [4]:
import tensorflow as tf
import json

# Update the model_path to point to your .keras file.
model_path = "gs://tf_models_cv/v1_Feb_09_2025.keras"

# Load the model
model = tf.keras.models.load_model(model_path)

# Print summary and configuration
print("=== Model Summary ===")
model.summary()

print("\n=== Model Configuration ===")
config = model.get_config()
print(json.dumps(config, indent=2))


=== Model Summary ===


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MobileNetV3Small (Functional)   │ (None, 7, 7, 576)      │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 576)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,125,007 (4.29 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,125,005 (4.29 MB)

 Optimizer params: 2 (12.00 B)


=== Model Configuration ===
{
  "name": "functional_1",
  "trainable": false,
  "layers": [
    {
      "module": "keras.layers",
      "class_name": "InputLayer",
      "config": {
        "batch_shape": [
          null,
          224,
          224,
          3
        ],
        "dtype": "float32",
        "sparse": false,
        "name": "input_layer"
      },
      "registered_name": null,
      "name": "input_layer",
      "inbound_nodes": []
    },
    {
      "module": "keras",
      "class_name": "Sequential",
      "config": {
        "name": "sequential",
        "trainable": true,
        "dtype": {
          "module": "keras",
          "class_name": "DTypePolicy",
          "config": {
            "name": "float32"
          },
          "registered_name": null
        },
        "layers": [
          {
            "module": "keras.layers",
            "class_name": "InputLayer",
            "config": {
              "batch_shape": [
                null,
              